In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd "/content/gdrive/My Drive/UNet_PyTorch_Implementation/"

/content/gdrive/My Drive/UNet_PyTorch_Implementation


In [3]:
! pip install albumentations==0.4.6

In [4]:
import os
import torch
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from utils import*
from dataset import dataset
from model import UNet

if __name__=="__main__":
    #Define somethings
    resized_height=128
    resized_width=128
    batchsize=32
    lr=1e-4
    device="cuda" if torch.cuda.is_available() else "cpu"
    print("Device",device)
    num_workers=4
    pin_memory=True
    epochs=3

    img_dir="./carvana_dataset/train/"
    mask_dir="./carvana_dataset/train_masks/"

    #Train, test split
    img_names=os.listdir(img_dir)
    train_img_names,val_img_names=train_test_split(img_names,test_size=0.1)
    print("num_train: ",len(train_img_names))
    print("num_val: ",len(val_img_names))

    #Data augmentation
    train_transform=get_transform(resized_height,resized_width,train=True)
    val_transform=get_transform(resized_height,resized_width,train=False)

    #Get datasets
    train_ds=dataset(img_dir,mask_dir,train_img_names,train_transform)
    val_ds=dataset(img_dir,mask_dir,val_img_names,val_transform)

    #Get dataloaders
    train_loader=DataLoader(train_ds,batch_size=batchsize,num_workers=num_workers,pin_memory=pin_memory,shuffle=True)
    val_loader=DataLoader(val_ds,batch_size=batchsize,num_workers=2,pin_memory=pin_memory,shuffle=True)

    #Get model
    model=UNet(in_channels=3,out_channels=1)

    #Loss function and optimizer
    loss_fn=nn.BCEWithLogitsLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=lr)

    for epoch in range(epochs):
        train_fn(model,train_loader,loss_fn,optimizer,device=device)
        checkpoint={
            "state_dict":model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        #test
        train_loss,train_accuracy,train_dice_score=test_fn(model,train_loader,loss_fn,device)
        val_loss,val_accuracy,val_dice_score = test_fn(model,val_loader,loss_fn,device)
        print("Epoch : {}/{}, Train Loss :{:.2f}, Val Loss :{:.2f}, Train acc :{:.2f}, Val acc :{:.2f}, Train dice score :{:.2f}, Val dice score :{:.2f}"
            .format(epoch+1,epochs,train_loss,val_loss,train_accuracy,val_accuracy,train_dice_score,val_dice_score))


Device cuda
num_train:  4579
num_val:  509


100%|██████████| 144/144 [17:53<00:00,  7.45s/it, loss=0.208]


=> Saving checkpoint


  0%|          | 0/144 [00:00<?, ?it/s]

Epoch : 1/3, Train Loss :0.21, Val Loss :0.21, Train acc :98.76, Val acc :98.82, Train dice score :0.97, Val dice score :0.97


...

=> Saving checkpoint


  0%|          | 0/144 [00:00<?, ?it/s]

Epoch : 2/3, Train Loss :0.16, Val Loss :0.16, Train acc :99.16, Val acc :99.19, Train dice score :0.98, Val dice score :0.98


100%|██████████| 144/144 [02:07<00:00,  1.13it/s, loss=0.123]


=> Saving checkpoint
Epoch : 3/3, Train Loss :0.12, Val Loss :0.12, Train acc :99.24, Val acc :99.24, Train dice score :0.98, Val dice score :0.98
